In [2]:
# Importa las bibliotecas necesarias
import os
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
import glob
import importlib
import butterfly_data_module
importlib.reload(butterfly_data_module)
from butterfly_data_module import ButterflyDataModule
from classifier import ClassifierA, ClassifierB1, ClassifierB2
from unet_autoencoder import UNetAutoencoder
import cv2
from dotenv import load_dotenv

print("GPU disponible:", torch.cuda.is_available())


GPU disponible: True


In [3]:
!python3 train_autoencoder.py

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: miguelsanchez712000 (miguelsanchez712000-itcr). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in ./wandb/run-20241118_201716-kvvmi6lx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run summer-serenity-16
wandb: ⭐️ View project at https://wandb.ai/miguelsanchez712000-itcr/butterfly_autoencoder
wandb: 🚀 View run at https://wandb.ai/miguelsanchez712000-itcr/butterfly_autoencoder/runs/kvvmi6lx
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params | Mode 
------------------------------------------------
0 | encoder1 | Sequential | 1.8 K  | train
1 | encoder2 | Sequential | 73.9 K | train
2 | encoder3 | Sequential | 295 K  

In [4]:
!python3 train_classifiers.py

/home/miguel/.local/lib/python3.10/site-packages/lightning_fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
GPU available: True (cuda), used: True
TPU available

In [3]:
# Cargar las variables de entorno del archivo .env
load_dotenv()
data_loader_dir = os.getenv("DATA_LOADER")

data_module = ButterflyDataModule(
    data_dir=data_loader_dir,
    batch_size=64,
    unlabeled_ratio=0.5,
    labeled_ratio=0.5    
)
data_module.setup()

In [16]:
data_checkpoints_dir = os.getenv("DATA_CHECKPOINTS")
autoencoder_dir = os.getenv("AUTOENCODER")

# Ruta al directorio de puntos de control
ckpt_dir = data_checkpoints_dir

# Cargar el autoencoder para obtener el encoder
try:
    autoencoder = UNetAutoencoder.load_from_checkpoint('/home/miguel/Desktop/IA_Proyecto3/autoencoder.ckpt')
    encoder = autoencoder.encoder
    print("Autoencoder cargado con éxito.")
except Exception as e:
    print("Error al cargar el Autoencoder:", e)

# Define la función para encontrar el último checkpoint
def find_latest_checkpoint(prefix):
    # Busca todos los archivos que comiencen con el prefijo proporcionado
    files = glob.glob(os.path.join(ckpt_dir, f"{prefix}-epoch=*-train_loss=*.ckpt"))
    if not files:
        raise FileNotFoundError(f"No se encontró ningún archivo que coincida con el prefijo: {prefix}")
    # Retorna el archivo más reciente basado en la fecha de modificación
    return max(files, key=os.path.getmtime)

# Función para cargar y evaluar un modelo
def evaluate_model(ckpt_path, model_class, num_classes=20, encoder=None):
    # Carga el modelo desde el punto de control
    if model_class in [ClassifierB1, ClassifierB2]:
        # Proporciona el encoder si la clase del modelo lo requiere
        model = model_class.load_from_checkpoint(ckpt_path, encoder=encoder, num_classes=num_classes)
    else:
        # Carga los modelos que no requieren un encoder
        model = model_class.load_from_checkpoint(ckpt_path, num_classes=num_classes)
    
    model.eval()  # Modo de evaluación

    # Crea un Trainer en modo de prueba
    trainer = pl.Trainer(accelerator='cuda', devices=1)

    # Evalúa el modelo en el conjunto de datos de prueba
    result = trainer.test(model, dataloaders=data_module.test_dataloader())
    return result


Autoencoder cargado con éxito.


In [17]:
data_loader_dir = os.getenv("DATA_LOADER")

# Evaluar con diferentes proporciones de datos etiquetados y no etiquetados
for labeled_ratio in [0, 0.25, 0.5, 0.75, 1]:
    unlabeled_ratio = 1 - labeled_ratio
    print(f"\nEvaluando con {labeled_ratio*100:.0f}% datos etiquetados y {unlabeled_ratio*100:.0f}% datos sin etiquetar")

    # Instancia el DataModule ajustado
    data_module = ButterflyDataModule(
        data_dir=data_loader_dir,
        batch_size=64,
        unlabeled_ratio=unlabeled_ratio,
        labeled_ratio=labeled_ratio
    )
    data_module.setup()

    # Evaluar ClassifierA
    try:
        ckpt_path_a = find_latest_checkpoint('classifier_a')
        print("Cargando ClassifierA desde:", ckpt_path_a)
        result_a = evaluate_model(ckpt_path_a, ClassifierA)
        print("Resultados de ClassifierA:", result_a)
    except FileNotFoundError as e:
        print(e)

    # Evaluar ClassifierB1
    try:
        ckpt_path_b1 = find_latest_checkpoint('classifier_b1')
        print("Cargando ClassifierB1 desde:", ckpt_path_b1)
        result_b1 = evaluate_model(ckpt_path_b1, ClassifierB1, encoder=encoder)
        print("Resultados de ClassifierB1:", result_b1)
    except FileNotFoundError as e:
        print(e)

    # Evaluar ClassifierB2
    try:
        ckpt_path_b2 = find_latest_checkpoint('classifier_b2')
        print("Cargando ClassifierB2 desde:", ckpt_path_b2)
        result_b2 = evaluate_model(ckpt_path_b2, ClassifierB2, encoder=encoder)
        print("Resultados de ClassifierB2:", result_b2)
    except FileNotFoundError as e:
        print(e)


Evaluando con 0% datos etiquetados y 100% datos sin etiquetar
Cargando ClassifierA desde: /home/miguel/Desktop/IA_Proyecto3/checkpoints/classifier_a-epoch=28-train_loss=0.00.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-11-19 19:54:40.804677: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 19:54:41.709325: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7760000228881836     │
│          test_f1          │    0.2694180905818939     │
│         test_loss         │    0.9313967823982239     │
│      test_precision       │    0.3047890067100525     │
│        test_recall        │    0.24378220736980438    │
└───────────────────────────┴───────────────────────────┘

Resultados de ClassifierA: [{'test_loss': 0.9313967823982239, 'test_acc': 0.7760000228881836, 'test_precision': 0.3047890067100525, 'test_recall': 0.24378220736980438, 'test_f1': 0.2694180905818939}]
Cargando ClassifierB1 desde: /home/miguel/Desktop/IA_Proyecto3/checkpoints/classifier_b1-epoch=03-train_loss=0.49.ckpt


...

Resultados de ClassifierB1: [{'test_loss': 0.8980317115783691, 'test_acc': 0.7200000286102295, 'test_precision': 0.2806912064552307, 'test_recall': 0.21938520669937134, 'test_f1': 0.23857617378234863}]
Cargando ClassifierB2 desde: /home/miguel/Desktop/IA_Proyecto3/checkpoints/classifier_b2-epoch=00-train_loss=1.38.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6200000047683716     │
│          test_f1          │    0.18256324529647827    │
│         test_loss         │    1.3382315635681152     │
│      test_precision       │    0.23783369362354279    │
│        test_recall        │     0.159159854054451     │
└───────────────────────────┴───────────────────────────┘

Resultados de ClassifierB2: [{'test_loss': 1.3382315635681152, 'test_acc': 0.6200000047683716, 'test_precision': 0.23783369362354279, 'test_recall': 0.159159854054451, 'test_f1': 0.18256324529647827}]
